### Plotting the Food Dollar Farm Share Over Time

This notebook compares the farm share and marketing share of the food dollar over time for both food-at-home and food-away-from-home purchases. Data were obtained from the USDA-ERS' [Food Dollar Series](https://www.ers.usda.gov/data-products/food-dollar-series/download-the-data).

Click the badge below to open in Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chuckgrigsby0/agec-370/blob/main/notebooks/05_plot_food_dollar_farm_share.ipynb)

We first import the necessary libraries and load the data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/chuckgrigsby0/agec-370/main/data/"

# Load monthly steer price data directly from GitHub URL
fd = pd.read_csv(base_url + 'food_dollar_real_ers.csv')


Next, we filter the data to include the `Food dollar`, `Food at home dollar`, and `Food away from home dollar` tables and the `Total` and `Farm share` categories.

#### Filter data

In [ ]:
farm_share_filter = ['Total', 'Farm share']
table_name_filter = ['Food dollar', 'Food at home dollar', 'Food away from home dollar']

fd_clean = fd.query(
    "(Table_name in @table_name_filter) & "
    "(Units == 'Cents per Domestic Real Food Dollar') & "
    "(Category_desc in @farm_share_filter)").reset_index(drop=True)

 ### Pivot to wide format

 We then pivot the data to obtain separate columns for `Total` and `Farm share`, and calculate the `Marketing` share as the difference between `Total` and `Farm share`. Note that we need to include both `Year` and `Table_name` in the index argument. 

In [ ]:
# Pivot the data wider so Total and Farm share become separate columns
fd_wide = fd_clean.pivot(index=['Year', 'Table_name'], columns='Category_desc', values='Total').reset_index()

# Calculate Marketing as Total - Farm share
fd_wide['Marketing'] = fd_wide['Total'] - fd_wide['Farm share']

### Melting the data for plotting

Finally, we re-pivot (melt) the data back into long format and convert cents to dollars before creating the line plot. 

In [ ]:
# Now pivot back to long format for plotting
fd_plot = fd_wide.melt(
    id_vars=['Year', 'Table_name'], 
    value_vars=['Farm share', 'Marketing'],  # Only include these two for plotting
    var_name='farm_share', 
    value_name='Value'
)

# This sorts the data by year and farm share category
fd_plot.sort_values(by=['Year', 'Table_name', 'farm_share'], inplace=True)

# Convert cents to dollars
fd_plot['Value'] = fd_plot['Value'] / 100  # Convert cents to dollars

We will first plot the farm share of the food dollar for both food-at-home and food-away-from-home purchases. The `!= 'Marketing'` filter is used to exclude the marketing share data.

In [ ]:
fd_plot_farm_share = fd_plot.query("farm_share != 'Marketing'")

### Figure of Farm Share for Total Food Dollor, FAH and FAFH

In [ ]:
# Set seaborn style
sns.set_style("whitegrid")

# Set figure size
plt.figure(figsize=(14, 7))

# Define color palette and dash styles
colors = {'Food dollar': '#2A9D8F', 'Food at home dollar': '#E76F51', 'Food away from home dollar': "#0E0868"}
dash_styles = {'Food dollar': (1,0), 'Food at home dollar': (1, 2), 'Food away from home dollar': (3, 1, 1, 1)}

# Create line plot
sns.lineplot(
    data=fd_plot_farm_share, 
    x='Year', 
    y='Value', 
    hue='Table_name',
    style='Table_name',
    hue_order=['Food dollar', 'Food at home dollar', 'Food away from home dollar'],
    style_order=['Food dollar', 'Food at home dollar', 'Food away from home dollar'],
    palette=colors,
    dashes=dash_styles,
    marker='o',           # Circular markers at each data point
    markersize=8,         # Slightly larger markers for visibility
    linewidth=2.5,        # Thicker lines for better visibility
    alpha=0.9             # Slight transparency 
)

# Enhance title with better formatting
plt.title(
    'Farm Share for Total Food Dollar, Food at Home, and Food Away from Home Expenditures (1993-2023)',
    fontsize=16, 
    fontweight='bold',
    pad=20                # Add padding above title
)

# Define axis labels
plt.ylabel('Cents per Dollar', fontsize=14, fontweight='semibold')
plt.xlabel('Year', fontsize=14, fontweight='semibold')

# Set x-axis to show every year and rotate labels 90 degrees
years = fd_plot_farm_share['Year'].unique()
plt.xticks(years, rotation=90, fontsize=12)

# Define y-axis tick labels
plt.yticks(fontsize=12)

# Define legend positioning and styling characteristics
plt.legend(
    title='Category',
    title_fontsize=13,
    fontsize=12,
    loc='best',           # Automatically find best position
    frameon=True,         # Add frame around legend
    shadow=True,          # Add subtle shadow
    fancybox=True         # Rounded corners on legend box
)

# Add a note about data source
plt.figtext(
    0.99, 0.01, 
    'Data Source: USDA Economic Research Service', 
    ha='right', 
    fontsize=9, 
    style='italic',
    alpha=0.7
)

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Display the plot
plt.show()

Next, we plot the marketing share of the food dollar, which equals `1 - Farm Share`, for both food-at-home and food-away-from-home purchases. 

In [ ]:
fd_plot_market_share = fd_plot.query("farm_share == 'Marketing'")

### Figure of Marketing Share for Total Food Dollor, FAH and FAFH

In [ ]:
# Set seaborn style
sns.set_style("whitegrid")

# Set figure size
plt.figure(figsize=(14, 7))

# Define color palette and dash styles
colors = {'Food dollar': '#2A9D8F', 'Food at home dollar': '#E76F51', 'Food away from home dollar': "#0E0868"}
dash_styles = {'Food dollar': (1,0), 'Food at home dollar': (1, 2), 'Food away from home dollar': (3, 1, 1, 1)}

# Create line plot
sns.lineplot(
    data=fd_plot_market_share, 
    x='Year', 
    y='Value', 
    hue='Table_name',
    style='Table_name',
    hue_order=['Food dollar', 'Food at home dollar', 'Food away from home dollar'],
    style_order=['Food dollar', 'Food at home dollar', 'Food away from home dollar'],
    palette=colors,
    dashes=dash_styles,
    marker='o',           # Circular markers at each data point
    markersize=8,         # Slightly larger markers for visibility
    linewidth=2.5,        # Thicker lines for better visibility
    alpha=0.9             # Slight transparency 
)

# Enhance title with better formatting
plt.title(
    'Marketing Share for Total Food Dollar, Food at Home, and Food Away from Home Expenditures (1993-2023)',
    fontsize=16, 
    fontweight='bold',
    pad=20                # Add padding above title
)

# Define axis labels
plt.ylabel('Cents per Dollar', fontsize=14, fontweight='semibold')
plt.xlabel('Year', fontsize=14, fontweight='semibold')

# Set x-axis to show every year and rotate labels 90 degrees
years = fd_plot_market_share['Year'].unique()
plt.xticks(years, rotation=90, fontsize=12)

# Define y-axis tick labels
plt.yticks(fontsize=12)

# Define legend positioning and styling characteristics
plt.legend(
    title='Category',
    title_fontsize=13,
    fontsize=12,
    loc='best',           # Automatically find best position
    frameon=True,         # Add frame around legend
    shadow=True,          # Add subtle shadow
    fancybox=True         # Rounded corners on legend box
)

# Add a note about data source
plt.figtext(
    0.99, 0.01, 
    'Data Source: USDA Economic Research Service', 
    ha='right', 
    fontsize=9, 
    style='italic',
    alpha=0.7
)

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Display the plot
plt.show()